In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

/aul/homes/rrang016/.conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data= pd.read_csv("data/final_concatenated_data.csv")
data 

,Unnamed: 0,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,0,10/16/2020,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,...,2.28,6.925,0.106,7.133,2.09,2.09,5.92,2.92,3.03,4.04
1,1,10/17/2020,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,...,0.01,7.025,0.000,7.133,3.87,3.87,5.97,2.95,3.08,4.05
2,2,10/18/2020,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,...,0.00,7.008,0.088,7.138,3.51,3.51,5.98,2.93,3.06,4.05
3,3,10/19/2020,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,...,0.11,6.986,0.256,7.167,1.98,1.98,5.98,2.91,3.05,4.06
4,4,10/20/2020,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,...,0.37,6.977,0.678,7.195,1.27,1.27,5.99,2.90,3.06,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,1406,8/22/2024,297.72,294.23,0.17,0.17,30.02,559.39,0.0,358,...,1.72,7.289,0.050,7.466,2.75,2.75,6.13,2.85,2.78,3.88
1407,1407,8/23/2024,240.47,293.74,0.45,0.45,0.29,491.92,0.0,375,...,0.04,7.360,0.000,7.447,2.75,2.75,6.18,2.90,2.81,3.88
1408,1408,8/24/2024,242.45,293.75,0.00,0.00,0.00,439.40,0.0,377,...,0.00,7.360,0.010,7.435,2.75,2.75,6.20,2.88,2.80,3.88
1409,1409,8/25/2024,286.80,294.24,0.00,0.00,0.00,470.49,0.0,380,...,0.00,7.345,0.270,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [4]:
# Convert 'date' column to proper datetime format
data['date'] = pd.to_datetime(data['date'])

# Verify the column types to confirm the 'date' is now datetime
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Unnamed: 0        1411 non-null   int64         
 1   date              1411 non-null   datetime64[ns]
 2   S199              1411 non-null   float64       
 3   S200              1411 non-null   float64       
 4   S332B             1411 non-null   float64       
 5   S332BN            1411 non-null   float64       
 6   S332C             1411 non-null   float64       
 7   S332D             1411 non-null   float64       
 8   S332DX1           1411 non-null   float64       
 9   S12A              1411 non-null   int64         
 10  S12B              1411 non-null   int64         
 11  S12C              1411 non-null   int64         
 12  S12D              1411 non-null   int64         
 13  S333              1411 non-null   float64       
 14  S333N             1411 n

In [5]:
# When displaying or exporting, you can format the date as MM/DD/YYYY
data['formatted_date'] = data['date'].dt.strftime('%m/%d/%Y')

# Print the first few rows to see the formatted date
print(data[['date', 'formatted_date']])                                                                                                    # Drop the 'formatted_date' column if not needed
data = data.drop(columns=['Unnamed: 0','formatted_date'])

           date formatted_date
0    2020-10-16     10/16/2020
1    2020-10-17     10/17/2020
2    2020-10-18     10/18/2020
3    2020-10-19     10/19/2020
4    2020-10-20     10/20/2020
...         ...            ...
1406 2024-08-22     08/22/2024
1407 2024-08-23     08/23/2024
1408 2024-08-24     08/24/2024
1409 2024-08-25     08/25/2024
1410 2024-08-26     08/26/2024

[1411 rows x 2 columns]


In [6]:
data['date'] = data['date'].dt.strftime('%m/%d/%Y')

# Check the data to confirm the index
data 

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,10/16/2020,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,382,...,2.28,6.925,0.106,7.133,2.09,2.09,5.92,2.92,3.03,4.04
1,10/17/2020,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,405,...,0.01,7.025,0.000,7.133,3.87,3.87,5.97,2.95,3.08,4.05
2,10/18/2020,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,394,...,0.00,7.008,0.088,7.138,3.51,3.51,5.98,2.93,3.06,4.05
3,10/19/2020,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,392,...,0.11,6.986,0.256,7.167,1.98,1.98,5.98,2.91,3.05,4.06
4,10/20/2020,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,399,...,0.37,6.977,0.678,7.195,1.27,1.27,5.99,2.90,3.06,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,08/22/2024,297.72,294.23,0.17,0.17,30.02,559.39,0.0,358,250,...,1.72,7.289,0.050,7.466,2.75,2.75,6.13,2.85,2.78,3.88
1407,08/23/2024,240.47,293.74,0.45,0.45,0.29,491.92,0.0,375,265,...,0.04,7.360,0.000,7.447,2.75,2.75,6.18,2.90,2.81,3.88
1408,08/24/2024,242.45,293.75,0.00,0.00,0.00,439.40,0.0,377,268,...,0.00,7.360,0.010,7.435,2.75,2.75,6.20,2.88,2.80,3.88
1409,08/25/2024,286.80,294.24,0.00,0.00,0.00,470.49,0.0,380,272,...,0.00,7.345,0.270,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [7]:
# Replace with your actual DataFrame
df = pd.DataFrame(data)

In [8]:
#Convert 'date' to datetime format if not 
df['date'] = pd.to_datetime(df['date'])

#Melt the DataFrame to long format
long_df = pd.melt(df, id_vars=['date'], 
                  value_vars=[col for col in df.columns if col != 'date'], 
                  var_name='unique_id', 
                  value_name='y')

#Rename the date column to 'ds'
long_df.rename(columns={'date': 'ds'}, inplace=True)

# Display the resulting long format DataFrame
print(long_df) 

              ds   unique_id       y
0     2020-10-16        S199  242.17
1     2020-10-17        S199  296.05
2     2020-10-18        S199  277.85
3     2020-10-19        S199  224.13
4     2020-10-20        S199  260.51
...          ...         ...     ...
52202 2024-08-22  NP62_stage    3.88
52203 2024-08-23  NP62_stage    3.88
52204 2024-08-24  NP62_stage    3.88
52205 2024-08-25  NP62_stage    3.98
52206 2024-08-26  NP62_stage    4.03

[52207 rows x 3 columns]


In [9]:
# Reorder columns
long_df = long_df[['unique_id', 'ds', 'y']]

long_df

,unique_id,ds,y
0,S199,2020-10-16,242.17
1,S199,2020-10-17,296.05
2,S199,2020-10-18,277.85
3,S199,2020-10-19,224.13
4,S199,2020-10-20,260.51
...,...,...,...
52202,NP62_stage,2024-08-22,3.88
52203,NP62_stage,2024-08-23,3.88
52204,NP62_stage,2024-08-24,3.88
52205,NP62_stage,2024-08-25,3.98


In [10]:
Y_df = long_df[['unique_id', 'ds', 'y']]                                                                                                                                                     # Rename the 'y' column to 'target'
Y_df = Y_df.rename(columns={'y': 'target'})
Y_df

,unique_id,ds,target
0,S199,2020-10-16,242.17
1,S199,2020-10-17,296.05
2,S199,2020-10-18,277.85
3,S199,2020-10-19,224.13
4,S199,2020-10-20,260.51
...,...,...,...
52202,NP62_stage,2024-08-22,3.88
52203,NP62_stage,2024-08-23,3.88
52204,NP62_stage,2024-08-24,3.88
52205,NP62_stage,2024-08-25,3.98


In [ ]:
n_time = len(Y_df.ds.unique())
val_size = int(0.15 * n_time)
test_size = int(0.15 * n_time)

In [ ]:
print('n_time=', n_time)
print('val_size', val_size)
print('test_size', test_size)

In [ ]:
#For the .fit method, model to be trained on provided dataset
Y_train_df = Y_df[Y_df['ds'] < Y_df['ds'].values[-422]]  # Training set
Y_val_df = Y_df[(Y_df['ds'] >= Y_df['ds'].values[-422]) & (Y_df['ds'] < Y_df['ds'].values[-211])]  # Validation set
Y_test_df = Y_df[Y_df['ds'] >= Y_df['ds'].values[-211]].reset_index(drop=True)

In [11]:
# Define parameters
history_window_size = 128  # The last 128 rows
prediction_length = 28  # Predict the next 28 days

In [12]:
# Initialize the predictions list
predictions_list = []

In [ ]:
# Loop over the date range
for current_date in pd.date_range('2024-01-29', '2024-08-26', freq='D'):
    print(f"Processing date: {current_date}")  # Monitor progress

    # Filter the data to include only the last 'history_window_size' days for each unique_id
    train_data = Y_df[Y_df['ds'] <= current_date]  # Filter rows up to the current date
    train_data = train_data.groupby('unique_id').tail(history_window_size)  # Take the last 'history_window_size' rows per ID

    # Convert to TimeSeriesDataFrame
    train_data_ts = TimeSeriesDataFrame.from_data_frame(
        train_data,
        id_column="unique_id",
        timestamp_column="ds",
    )

    # Train-test split
    train_data, test_data = train_data_ts.train_test_split(prediction_length)

    # Loop through each unique_id and fit the predictor
    for unique_id in train_data_ts.index.get_level_values('item_id').unique():
        print(f"  Processing unique_id: {unique_id},  ds: {current_date}")  # Monitor progress

        # Filter data for the current unique_id
        unique_train_data = train_data_ts[train_data_ts.index.get_level_values('item_id') == unique_id]
        
        # Initialize or update the predictor (fit with new data)
        predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(
            unique_train_data, presets="bolt_small",
        )

        # Generate predictions
        forecast = predictor.predict(unique_train_data)
        
        # Extract the 28th day prediction
        forecast_28th_day = forecast['mean'].iloc[-1]  # Get the 28th day forecast from the prediction output
        
        # Append the 28th day prediction to the list
        predictions_list.append({
            'unique_id': unique_id,
            'ds': current_date,
            'forecast_28th_day': forecast_28th_day
        })

In [ ]:
predictions_list

In [ ]:
df_predictions = pd.DataFrame(predictions_list)
df_predictions = df_predictions.rename(columns={'forecast_28th_day': 'Chronos'})
df_predictions

In [ ]:
# Sort the DataFrame by unique_id and ds to group the data properly
predictions_df = df_predictions.sort_values(by=['unique_id', 'ds']).reset_index(drop=True)

In [ ]:
predictions_df

In [ ]:
results_df = pd.merge(predictions_df, Y_test_df, on=['unique_id', 'ds'], how='inner')
results_df

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# List of stages to evaluate
stages = ['NP205_stage', 'P33_stage','G620_water_level', 'NESRS1', 'NESRS2']

for stage in stages:
    # Filter the data for the specific stage
    stage_data = results_df[results_df['unique_id'] == stage]
    print(f"======== {stage} ========")
    
    # Extract the actual and predicted values
    true = stage_data['target'].values
    pred = stage_data['Chronos'].values
    
    # Calculate MAE and RMSE
    mae_value = mae(true, pred)
    rmse_value = np.sqrt(mse(true, pred))
    
    # Print the results
    print(f'MAE = {mae_value:.3f}')
    print(f'RMSE = {rmse_value:.3f}\n')

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# List of stages to evaluate
stages = ['NP205_stage', 'P33_stage', 'G620_water_level', 'NESRS1', 'NESRS2']

# List of models (these correspond to column names in the DataFrame)
models = ['Chronos']

# Dictionary to store overall MAE and RMSE for each model
overall_mae = {model: [] for model in models}
overall_rmse = {model: [] for model in models}

# Iterate through each stage
for stage in stages:
    # Filter the data for the specific stage
    stage_data = results_df[results_df['unique_id'] == stage]
    
    # Check if data for the stage exists
    if stage_data.empty:
        print(f"No data found for {stage}. Skipping...\n")
        continue
    
    # Extract the actual values
    true = stage_data['target'].values
    
    # Iterate through each model
    for model in models:
        # Extract the predicted values for the model
        pred = stage_data[model].values
        
        # Calculate MAE and RMSE, then append to the corresponding model's list
        model_mae = mae(true, pred)
        model_rmse = np.sqrt(mse(true, pred))
        
        overall_mae[model].append(model_mae)
        overall_rmse[model].append(model_rmse)

# Calculate the overall MAE and RMSE (mean of MAEs and RMSEs across the 5 stations) for each model
final_overall_mae = {model: np.mean(maes) for model, maes in overall_mae.items()}
final_overall_rmse = {model: np.mean(rmses) for model, rmses in overall_rmse.items()}

# Print the overall MAE and RMSE for each model
print("Overall MAE for Chronos(only for selected stages):")
for model, mae_value in final_overall_mae.items():
    print(f"{model}: MAE = {mae_value:.3f}")

print("\nOverall RMSE for Chronos(only for selected stages):")
for model, rmse_value in final_overall_rmse.items():
    print(f"{model}: RMSE = {rmse_value:.3f}")

In [ ]:
# Save predictions to a CSV file
df_predictions.to_csv('Output/Chronos-Predictions-28days.csv', index=False)